## LowStar Code

In [ ]:
import os
import numpy as np
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt
from TDA_Image_Process2 import topological_process_img

# Define the directories
image_dir = "./HAM10000_images_part_1/"
mask_dir = "./HAM10000_segmentations_lesion_tschandl/"

# Get list of all images
image_files = [f for f in os.listdir(image_dir) if f.endswith('.jpg')]

# Split dataset for testing
np.random.seed(42)  
np.random.shuffle(image_files)
split_idx = int(len(image_files) * 0.01)  # 50 for testing
test_files = image_files[:split_idx]

def calculate_metrics(pred_mask, gt_mask):
    # Convert masks to binary
    pred_mask = pred_mask.astype(bool)
    gt_mask = gt_mask.astype(bool)
    
    # Calculate True Positives, False Positives, True Negatives, False Negatives
    TP = np.logical_and(pred_mask, gt_mask).sum()
    TN = np.logical_and(~pred_mask, ~gt_mask).sum()
    FP = np.logical_and(pred_mask, ~gt_mask).sum()
    FN = np.logical_and(~pred_mask, gt_mask).sum()
    
    # Calculate IOU
    intersection = np.logical_and(gt_mask, pred_mask)
    union = np.logical_or(gt_mask, pred_mask)
    iou = np.sum(intersection) / np.sum(union) if np.sum(union) > 0 else 0.0
    
    # Calculate Pixel Accuracy
    pixel_accuracy = (TP + TN) / float(TP + TN + FP + FN)
    
    return iou, pixel_accuracy


ious = []
accuracies = []

# Process test images
for img_file in tqdm(test_files):
    try:
        # Get corresponding mask filename
        mask_file = img_file.replace('.jpg', '_segmentation.png')
        
        # Read image and mask
        img_path = os.path.join(image_dir, img_file)
        mask_path = os.path.join(mask_dir, mask_file)
        
        if not os.path.exists(mask_path):
            print(f"Mask not found for {img_file}")
            continue
            
        # Read and process image
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        img_normalized = img_gray.astype(float) / 255.0
        
        # Read ground truth mask and ensure binary
        gt_mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        gt_mask = cv2.resize(gt_mask, (img_gray.shape[1], img_gray.shape[0]))
        gt_mask = (gt_mask > 128).astype(np.uint8)  # Convert to binary
        
        # Calculate parameters
        window_size = int(np.sqrt(img_gray.shape[0] * img_gray.shape[1]) / 25)
        border_width = int(np.sqrt(img_gray.shape[0] * img_gray.shape[1]) / 100)
        
        # Apply TDA processing for segmentation
        result = topological_process_img(img_normalized, 
                                       window_size=window_size,
                                       border_width=border_width)
        
        # Get segmentation mask and ensure binary
        pred_mask = result['components']
        pred_mask = (pred_mask > 0).astype(np.uint8)  # Convert to binary
        
        # Calculate metrics
        iou, pixel_acc = calculate_metrics(pred_mask, gt_mask)
        ious.append(iou)
        accuracies.append(pixel_acc)
        
        # Display results
        plt.clf()
        fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(20, 5))
        
        # Original image
        ax1.imshow(img)
        ax1.set_title('Original Image')
        ax1.axis('off')
        
        # Ground truth mask
        ax2.imshow(gt_mask, cmap='gray')
        ax2.set_title('Ground Truth')
        ax2.axis('off')
        
        # Predicted mask
        ax3.imshow(pred_mask, cmap='gray')
        ax3.set_title('TDA Prediction')
        ax3.axis('off')
        
        # Overlap visualization
        overlap = np.zeros_like(img)
        overlap[..., 0] = gt_mask * 255  # Ground truth in red channel
        overlap[..., 1] = pred_mask * 255  # Prediction in green channel
        ax4.imshow(overlap)
        ax4.set_title('Overlap\nRed: Ground Truth\nGreen: Prediction')
        ax4.axis('off')
        
        plt.suptitle(f"Processed: {img_file}\nIOU: {iou:.3f}, Pixel Accuracy: {pixel_acc:.3f}")
        plt.tight_layout()
        plt.show()
        
    except Exception as e:
        print(f"Error processing {img_file}: {str(e)}")
        continue


if ious: 
    print(f"\nProcessing complete! Processed {len(ious)} images successfully")
    print(f"Average IOU: {np.mean(ious):.5f}")
    print(f"Average Pixel Accuracy: {np.mean(accuracies):.5f}")
else:
    print("\nNo images were processed successfully")

In [ ]:
import os
import numpy as np
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt
from TDA_Process_GUDHI import topological_process_img

# Define the directories
image_dir = "./HAM10000_images_part_1/"
mask_dir = "./HAM10000_segmentations_lesion_tschandl/"

# Get list of all images
image_files = [f for f in os.listdir(image_dir) if f.endswith('.jpg')]

# Split dataset for testing
np.random.seed(42)  
np.random.shuffle(image_files)
split_idx = int(len(image_files) * 0.01)  # 50 images for testing
test_files = image_files[:split_idx]

def calculate_metrics(pred_mask, gt_mask):
    
    # Convert masks to binary
    pred_mask = pred_mask.astype(bool)
    gt_mask = gt_mask.astype(bool)
    
    # Calculate True Positives, False Positives, True Negatives, False Negatives
    TP = np.logical_and(pred_mask, gt_mask).sum()
    TN = np.logical_and(~pred_mask, ~gt_mask).sum()
    FP = np.logical_and(pred_mask, ~gt_mask).sum()
    FN = np.logical_and(~pred_mask, gt_mask).sum()
    
    # Calculate IOU
    intersection = np.logical_and(gt_mask, pred_mask)
    union = np.logical_or(gt_mask, pred_mask)
    iou = np.sum(intersection) / np.sum(union) if np.sum(union) > 0 else 0.0
    
    # Calculate Pixel Accuracy
    pixel_accuracy = (TP + TN) / float(TP + TN + FP + FN)
    
    return iou, pixel_accuracy


ious = []
accuracies = []

# Process test images
for img_file in tqdm(test_files):
    try:
        # Get corresponding mask filename
        mask_file = img_file.replace('.jpg', '_segmentation.png')
        
        # Read image and mask
        img_path = os.path.join(image_dir, img_file)
        mask_path = os.path.join(mask_dir, mask_file)
        
        if not os.path.exists(mask_path):
            print(f"Mask not found for {img_file}")
            continue
            
        # Read and process image
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        img_normalized = img_gray.astype(float) / 255.0
        
        # Read ground truth mask and ensure binary
        gt_mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        gt_mask = cv2.resize(gt_mask, (img_gray.shape[1], img_gray.shape[0]))
        gt_mask = (gt_mask > 128).astype(np.uint8)  # Convert to binary
        
        # Calculate parameters
        window_size = int(np.sqrt(img_gray.shape[0] * img_gray.shape[1]) / 25)
        border_width = int(np.sqrt(img_gray.shape[0] * img_gray.shape[1]) / 100)
        
        # Apply TDA processing for segmentation
        result = topological_process_img(img_normalized, 
                                       window_size=window_size,
                                       border_width=border_width)
        
        # Get segmentation mask 
        pred_mask = result['components']
        pred_mask = (pred_mask > 0).astype(np.uint8)  # Convert to binary
        
        # Calculate metrics
        iou, pixel_acc = calculate_metrics(pred_mask, gt_mask)
        ious.append(iou)
        accuracies.append(pixel_acc)
        
        # Display results
        plt.clf()
        fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(20, 5))
        
        # Original image
        ax1.imshow(img)
        ax1.set_title('Original Image')
        ax1.axis('off')
        
        # Ground truth mask
        ax2.imshow(gt_mask, cmap='gray')
        ax2.set_title('Ground Truth')
        ax2.axis('off')
        
        # Predicted mask
        ax3.imshow(pred_mask, cmap='gray')
        ax3.set_title('TDA Prediction')
        ax3.axis('off')
        
        # Overlap visualization
        overlap = np.zeros_like(img)
        overlap[..., 0] = gt_mask * 255  # Ground truth in red channel
        overlap[..., 1] = pred_mask * 255  # Prediction in green channel
        ax4.imshow(overlap)
        ax4.set_title('Overlap\nRed: Ground Truth\nGreen: Prediction')
        ax4.axis('off')
        
        plt.suptitle(f"Processed: {img_file}\nIOU: {iou:.3f}, Pixel Accuracy: {pixel_acc:.3f}")
        plt.tight_layout()
        plt.show()
        
    except Exception as e:
        print(f"Error processing {img_file}: {str(e)}")
        continue

# Print average metrics
if ious: 
    print(f"\nProcessing complete! Processed {len(ious)} images successfully")
    print(f"Average IOU: {np.mean(ious):.5f}")
    print(f"Average Pixel Accuracy: {np.mean(accuracies):.5f}")
else:
    print("\nNo images were processed successfully")